TAREA 5
-  Alexander Noblejas 20191966
-  Fernanda Pizarro 20191460

In [37]:
#instalación
!pip install pulp

In [46]:
import pulp as pp

Ejercicio 1: el problema de la dieta

In [47]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vTSq9X74urGAB_5n_MIJ9ZGIboKSvBdokVTBXVLh_qqZnmLRTJioOF431Rzys3Qi9UaFwWXjeq6Wmd5/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

In [48]:
#1. Nombre y declarar si es del tipo de problema de maximización o minimización
model = pp.LpProblem(name='diet-problem', # just the name
                     sense=pp.LpMinimize) # type of problem

In [49]:
#2. Declarar las VARIABLES
#¿Cuánto de cada elemento?
VegaV = pp.LpVariable(name="VegaVita",  # just the name
                    lowBound=0,  # ensure non-negativity
                    cat='Continuous') # here: you accept decimal values

HappyH = pp.LpVariable(name="HappyHealth",
                 lowBound=0,
                 cat='Continuous')

In [50]:
#3. Crear función de optimización
VegaCoeff=0.2
HappyCoeff=0.3
obj_func = VegaCoeff*VegaV + HappyCoeff*HappyH

In [51]:
#4.Representar las limitaciones
# SUBJECT TO:
C1= pp.LpConstraint(name='Vitamin C Constraint',   # just the name
                    e= 20*VegaV + 30*HappyH, rhs=60, # linear combination of constraint and rhs
                    sense=pp.LpConstraintGE)
C2= pp.LpConstraint(name='Calcium Constraint',
                    e= 500*VegaV + 250*HappyH, rhs=1000,
                    sense=pp.LpConstraintGE)
C3= pp.LpConstraint(name='Iron Constraint',
                    e= 9*VegaV + 2*HappyH, rhs=18,
                    sense=pp.LpConstraintGE, )
C4= pp.LpConstraint(name='Niacin Constraint',
                    e= 2*VegaV + 10*HappyH, rhs=20,
                    sense=pp.LpConstraintGE, )
C5= pp.LpConstraint(name='Magnesium Constraint',
                    e= 60*VegaV + 90*HappyH, rhs=360,
                    sense=pp.LpConstraintGE, )

In [52]:
model += obj_func
model += C1
model += C2
model += C3
model += C4
model += C5

In [53]:
solverToUse = pp.COIN_CMD(msg=False)
model.solve();

In [54]:
import pandas as pd

Results={"Model Status":pp.LpStatus[model.status]}
Results.update({"Optimal Solution":pp.value(model.objective)})
Results.update({v.name: v.varValue for v in model.variables()})
Results

{'Model Status': 'Optimal',
 'Optimal Solution': 1.2,
 'HappyHealth': 3.1304348,
 'VegaVita': 1.3043478}

In [55]:
#or
pd.DataFrame.from_dict(Results,orient='index').T.set_index('Model Status').style.format('{:,}')

,Optimal Solution,HappyHealth,VegaVita
Model Status,,,
Optimal,1.2,3.1304348,1.3043478


EJERCICIO 2
Desidiendo el destino de maestría

In [56]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vR7GL_wF1eKRO0JgEUyIx5cxXUhTQ8ZM4F3AE1MLr7GYG33dwEobrLo6O2MaV2d7Cv47TaTgHghkhrV/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

In [17]:
#pip install ahp

Note: you may need to restart the kernel to use updated packages.


In [23]:
# calling the function
from ahp.ahp import AHP

In [26]:
import pandas as pd

pairwise_cost=pd.read_excel("ahp2_tables.xlsx",sheet_name='Cost of living', index_col=0)
pairwise_language=pd.read_excel("ahp2_tables.xlsx",sheet_name='Language difficulty', index_col=0)
pairwise_possibilities=pd.read_excel("ahp2_tables.xlsx",sheet_name='Possibilities to get a job', index_col=0)
pairwise_criteria=pd.read_excel("ahp2_tables.xlsx",sheet_name='Criteria', index_col=0)

In [27]:
pairwise_criteria

,Cost of living,Language difficulty,Possibilities to get a job
Cost of living,1.000000,3,3.000000
Language difficulty,0.333333,1,0.333333
Possibilities to get a job,0.333333,3,1.000000


In [28]:
# follow the same order as the criteria
data = [pairwise_cost.values.tolist(),
        pairwise_language.values.tolist(),
        pairwise_possibilities.values.tolist(),
        pairwise_criteria.values.tolist()]

In [29]:
ahp = AHP()
ahp.fit(data)

In [30]:
Solution=pd.DataFrame(index=pairwise_possibilities.index)
Solution['Score']=ahp.classificate()
Solution['Ranking']=Solution.Score.rank(ascending=False)
Solution

,Score,Ranking
Brazil,0.060735,5.0
Spain,0.134003,4.0
USA,0.217910,2.0
England,0.377809,1.0
France,0.209543,3.0
